### Recalling narratives with retrieval augmented generation

How do hippocampus and neocortex work together to recall narratives (and other sequences), whilst minimising the amount of detail stored in the hippocampus?

* Neocortex creates gist
* Gist plus unpredictable details stored in HPC
* Stimulus triggers recall
* Relevant event(s) retrieved from HPC
* NC elaborates details

In [ ]:
!pip install openai --upgrade
!pip install llama-index --upgrade

In [8]:
import sys
sys.path.append('../data/')

import os
from openai import OpenAI
import pandas as pd
import random
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Document
import logging
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.prompts import PromptTemplate

logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

client = OpenAI(
    api_key='ADD YOUR KEY'
)

os.environ['OPENAI_API_KEY'] = 'ADD YOUR KEY'

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo"
)

chat_completion.choices[0].message.content

'This is a test.'

In [9]:
def get_output(input_text):
    completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": input_text,
            }
        ],
        model="gpt-3.5-turbo"
    )
    output_text = completion.choices[0].message.content
    return output_text

get_output("Hello world")

'Hello! How can I assist you today?'

In [10]:
def get_stories():
    df = pd.read_csv('../data/stories_train.csv')
    df['combined'] = df[[f'sentence{i}' for i in range(1,6)]].astype(str).agg(' '.join, axis=1)
    return df['combined'].tolist()

stories = get_stories()

In [11]:
stories[1]

"Tom had a very short temper. One day a guest made him very angry. He punched a hole in the wall of his house. Tom's guest became afraid and left quickly. Tom sat on his couch filled with regret about his actions."

In [12]:
get_output(f"{stories[1]} Brief summary:")

"Tom's short temper led him to punch a hole in his wall after getting angry at a guest, who then quickly left out of fear. Tom later regretted his actions."

In [13]:
gists = [get_output(f"{story} One line summary:") for story in stories[0:10]]
docs = [Document(text=txt) for txt in gists]
index = VectorStoreIndex.from_documents(docs)

In [14]:
for i in range(0,10):
    print("Event:")
    print(stories[i])
    print("Neocortex-generated gist:")
    print(gists[i])
    print("-----------")

Event:
David noticed he had put on a lot of weight recently. He examined his habits to try and figure out the reason. He realized he'd been eating too much fast food lately. He stopped going to burger places and started a vegetarian diet. After a few weeks, he started to feel much better.
Neocortex-generated gist:
David made a positive change in his diet and started feeling better after cutting out fast food and switching to a vegetarian diet.
-----------
Event:
Tom had a very short temper. One day a guest made him very angry. He punched a hole in the wall of his house. Tom's guest became afraid and left quickly. Tom sat on his couch filled with regret about his actions.
Neocortex-generated gist:
Tom let his short temper get the best of him, causing him to punch a hole in the wall in a fit of anger.
-----------
Event:
Marcus needed clothing for a business casual event. All of his clothes were either too formal or too casual. He decided to buy a pair of khakis. The pair he bought fit hi

In [15]:
# define custom retriever
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=1)

# define response synthesizer
response_synthesizer = get_response_synthesizer()

# vector query engine
query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=response_synthesizer,
)

qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information (and extrapolating from it), answer the query in detail.\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

Example recall 1:

In [16]:
response = query_engine.query("What happened when Melody's parents surprised her (in detail)?")

In [17]:
print("Original event:")
print(stories[5])
print("\nGist retrieved from hippocampus:")
print(response.source_nodes[0].text)
print("\nNeocortex-elaborated answer:")
print(response.response)

Original event:
Melody's parents surprised her with a trip to the big aquarium. Melody took a nap during the two hour car ride to the aquarium. When they arrived, Melody was energetic and excited. At the aquarium Melody saw sharks, tropical fish and many others. After five hours at the aquarium, Melody and her family drove home.

Gist retrieved from hippocampus:
Melody was surprised with a trip to the aquarium and had an exciting day exploring the different exhibits.

Neocortex-elaborated answer:
When Melody's parents surprised her with a trip to the aquarium, she was filled with excitement and joy. They spent the day exploring the various exhibits, marveling at the different sea creatures and learning about marine life. Melody was thrilled to see colorful fish, playful dolphins, graceful sea turtles, and majestic sharks up close. She eagerly listened to the educational talks and interactive demonstrations, soaking in all the information about the ocean and its inhabitants. The experie

Example recall 2:

In [18]:
response = query_engine.query("What happened to John the pastor (in detail)?")

In [19]:
print("Original event:")
print(stories[4])
print("\nGist retrieved from hippocampus:")
print(response.source_nodes[0].text)
print("\nNeocortex-elaborated answer:")
print(response.response)

Original event:
John was a pastor with a very bad memory. He tried to memorize his sermons many days in advance but to no avail. He decided to learn to sing to overcome his handicap. He then made all his sermons into music and sang them on Sundays. His congregation was delighted and so was he.

Gist retrieved from hippocampus:
Pastor with bad memory learns to sing sermons and delights congregation.

Neocortex-elaborated answer:
John, the pastor with a bad memory, faced challenges in delivering his sermons due to his forgetfulness. However, he found a creative solution by learning to sing his sermons instead. This not only helped him remember the content of his sermons but also delighted his congregation. By incorporating music into his sermons, John was able to engage his audience in a new and unique way, making his messages more memorable and impactful. This adaptation not only improved John's ability to deliver sermons effectively but also brought joy and inspiration to those who lis